In [8]:

import cv2                
import numpy as np
import matplotlib.pyplot as plt    
import random

#from extract_bottleneck_features import *
from glob import glob

from keras.applications.resnet50 import decode_predictions, preprocess_input, ResNet50
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing import image                  

from sklearn.datasets import load_files       

from PIL import ImageFile
from tqdm import tqdm

def extract_Xception(tensor):
    from keras.applications.xception import Xception, preprocess_input
    return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

%matplotlib inline

print("All revv'ed up and ready to go!")


All revv'ed up and ready to go!


In [20]:
# define function to load train, test, and validation datasets
NUM_CATEGORIES = 3

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), NUM_CATEGORIES)
    return files, targets

# load train, test, and validation datasets
base_folder = 'data'
valid_files, valid_targets = load_dataset('{}/valid'.format(base_folder))

# load list of disease names
disease_names = [item.split("/")[2] for item in sorted(glob("{}/valid/*/".format(base_folder)))]


In [23]:
#for item in sorted(glob("{}/valid/*/".format(base_folder))):
#    print()

print(disease_names)

print('Examples of file {} and target {}'.format(valid_files[-5:], valid_targets[-5:]))

#print(valid_files[-5:])
#print(valid_targets[-5:])

print('There are %d total disease categories.' % len(disease_names))
print('There are %d validation disease images.' % len(valid_files))


['melanoma', 'nevus', 'seborrheic_keratosis']
Examples of file ['data/valid/melanoma/ISIC_0013644.jpg' 'data/valid/nevus/ISIC_0015443.jpg'
 'data/valid/nevus/ISIC_0012313.jpg'
 'data/valid/seborrheic_keratosis/ISIC_0012720.jpg'
 'data/valid/nevus/ISIC_0007332.jpg'] and target [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
There are 3 total disease categories.
There are 150 validation disease images.


In [ ]:

# These are separated as I am using my computer to setup the framework, and AWS for training
# To save space, I didn't download train/test, a total of 10GB as zip!

train_files, train_targets = load_dataset('{}/train'.format(base_folder))
test_files, test_targets = load_dataset('{}/test'.format(base_folder))

print('There are %s total disease images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training disease images.' % len(train_files))
print('There are %d test disease images.'% len(test_files))


In [ ]:

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [ ]:
                           
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

print("All done, be sure to run this just once (per Jupyter notebook session)!")


In [ ]:

bottleneck_features = np.load('bottleneck_features/DogXceptionData.npz')

train_Exception = bottleneck_features['train']
valid_Exception = bottleneck_features['valid']
test_Exception = bottleneck_features['test']


Exception_model = Sequential()
Exception_model.add(GlobalAveragePooling2D(input_shape = train_Exception.shape[1:]))
Exception_model.add(Dense(len(dog_names), activation = 'softmax'))

Exception_model.summary()

Exception_model.compile(loss = 'categorical_crossentropy'
                        , metrics = ['accuracy']
                        , optimizer = 'rmsprop')

Exception_model.compile(loss = 'categorical_crossentropy'
                        , metrics = ['accuracy']
                        , optimizer = 'rmsprop')


In [ ]:

best_exception_model_file = 'saved_models/weights.best.Exception.hdf5'
    
checkpointer = ModelCheckpoint(filepath = best_inception_model_file
                               , save_best_only = True
                               , verbose = 1)

Exception_model.fit(train_InceptionV3, train_targets 
                    , batch_size = 20
                    , callbacks = [checkpointer]
                    , epochs = 20
                    , validation_data = (valid_Exception, valid_targets)
                    , verbose = 1
                   )


In [ ]:

Exception_model.load_weights(best_inception_model_file)

# get index of predicted dog breed for each image in test set
Exception_predictions = [np.argmax(Exception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Exception]

# report test accuracy
test_accuracy = 100*np.sum(np.array(Exception_predictions)==np.argmax(test_targets, axis=1))/len(Exception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)


In [ ]:

def disease(image_path):
    img_tensor = path_to_tensor(image_path)
    feature = extract_InceptionV3(img_tensor)
    #print(feature)
    #print(feature.shape)
    doggo_index = np.argmax(InceptionV3_model.predict(feature)) #np.expand_dims(feature, axis=0)))
    
    return dog_names[doggo_index]

disease('./data/valid/melanoma/ISIC_0012099.jpg')
disease('./data/valid/nevus/ISIC_0001769.jpg')
disease('./data/valid/seborrheic_keratosis/ISIC_0012143.jpg')
